In [10]:
# imports
import azureml.core

In [11]:
# assign current workspace
from azureml.core import Workspace
ws = Workspace.from_config()
print('Current workspace:', ws.name)

Current workspace: ml-workspace


In [12]:
# assign datastore
ds = ws.get_default_datastore()
print('Current datastore:', ds.name)

Current datastore: workspaceblobstore


In [20]:
# upload the file to blob
ds.upload_files(files=['./data/heart.csv'], target_path='data-heart/', overwrite=True, show_progress=True)

Uploading an estimated of 1 files
Uploading ./data/heart.csv
Uploaded ./data/heart.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_39dde4d9e65f4c928ee8a8ff32f49928

In [21]:
# register the dataset
from azureml.core import Dataset

# name of our dataset
ds_name = 'heart dataset'

if 'heart dataset' not in ws.datasets:
    # create tabular dataset from the data
    heart_data = Dataset.Tabular.from_delimited_files(path=(ds, 'data-heart/*.csv'))

    # register the dataset
    try:
        heart_data = heart_data.register(workspace=ws,
                                         name = ds_name,
                                         description= 'Heart Attach Data',
                                         tags = {'format' : 'csv'},
                                         create_new_version = True)
        print('Dataset %s registered with version %i.'%(heart_data.name, str(heart_data.version)))
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


Dataset already registered.


In [23]:
# Create a local folder for the pipeline step files

import os
experiment_folder = './heart_pipeline'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder)

./heart_pipeline


In [71]:
# %%writefile $experiment_folder/prep_diabetes.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Arg inputs
#ds_name
save_folder = './data/'

print('Loading Data...')
heart_df = Dataset.get_by_name(ws, ds_name).to_pandas_dataframe()

# Drop NAs
heart_df = heart_df.dropna()

# Modifying prediction label
heart_df.rename(columns={'output' : 'heart_attack'}, inplace=True)

# Change sex categorical feature to dummies
sex_type = pd.get_dummies(data=heart_df['sex'])
sex_type.columns = ['Male', 'Female']
chest_pain = pd.get_dummies(data=heart_df['cp'])
chest_pain.columns = ['Chest Pain 1', 'Chest Pain 2', 'Chest Pain 3',
                      'Chest Pain 4']
ex_angina = pd.get_dummies(data=heart_df['exng'])
ex_angina.columns = ['Exercise Angina: No', 'Exercise Angina: Yes']
slp_type = pd.get_dummies(data=heart_df['slp'])
slp_type.columns = ['Slope 1', 'Slope 2', 'Slope 3']
caa_type = pd.get_dummies(data=heart_df['caa'])
caa_type.columns = ['CAA 1', 'CAA 2', 'CAA 3', 'CAA 4', 'CAA 5']
thall_type = pd.get_dummies(data=heart_df['thall'])
thall_type.columns = ['Thall 1', 'Thall 2', 'Thall 3', 'Thall 4']


# Joining and removing modified columns
heart_df = pd.concat([sex_type, chest_pain, ex_angina, slp_type,
                      caa_type, thall_type, heart_df],
                     axis=1)
heart_df.drop(labels=['sex', 'cp', 'exng', 'slp', 'caa', 'thall'], 
              axis=1, inplace=True)

# Normalize numeric columns
scaler = MinMaxScaler()
norm_cols = ['age', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
                'oldpeak']
heart_df[norm_cols] = scaler.fit_transform(heart_df[norm_cols])

# Save the prepped data
print('Saving Data...')
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder, 'heartdata_norm.csv')
heart_df.to_csv(save_path, index=False, header=True)


Loading Data...
Saving Data...


In [72]:
# training model

In [ ]:
%%writefile $experiment_folder/prep_diabetes.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(diabetes))
run.log('raw_rows', row_count)

# remove nulls
diabetes = diabetes.dropna()

# Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
diabetes[num_cols] = scaler.fit_transform(diabetes[num_cols])

# Log processed rows
row_count = (len(diabetes))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
diabetes.to_csv(save_path, index=False, header=True)

# End the run
run.complete()

In [50]:
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler


# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='
parser.add_argument('--prepped-data', type=str, dest
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['raw_data'].to_pandas_

# Log raw row count
row_count = (len(diabetes))
run.log('raw_rows', row_count)

# remove nulls
diabetes = diabetes.dropna()

# Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','Diastolic diabetes[num_cols] = scaler.fit_transform(diabetes[n

# Log processed rows
row_count = (len(diabetes))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
diabetes.to_csv(save_path, index=False, header=True)

# # Get experiment run context
# run = Run.get_context()

# # Load the data
# print('Loading Data...')
# #heart = 
# heart = Dataset.get_by_name(ws, ds_name)

Loading Data...


In [55]:
heart.to_pandas_dataframe()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [ ]:
%%writefile $experiment_folder/prep_diabetes.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(diabetes))
run.log('raw_rows', row_count)

# remove nulls
diabetes = diabetes.dropna()

# Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
diabetes[num_cols] = scaler.fit_transform(diabetes[num_cols])

# Log processed rows
row_count = (len(diabetes))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
diabetes.to_csv(save_path, index=False, header=True)

# End the run
run.complete()